In [ ]:
一、寻找基础镜像

In [ ]:
1.找到基础镜像
（1）官方镜像仓库
    Docker Hub（默认仓库）：访问 https://hub.docker.com 直接搜索，例如：
        ubuntu：官方操作系统镜像
        python：Python 官方运行时
        nginx：Nginx 服务器镜像
（2）命令行搜索
    docker search ubuntu
    - 结果字段说明：OFFISICAL-标记是否为官方维护镜像（推荐优先选择）；STARS-镜像受欢迎程度
（3）根据已知镜像，谷歌搜索镜像网址

In [ ]:
2. 镜像选择标准
类型                   示例                    适用场景
操作系统镜像         ubuntu:20.04          需要完全控制环境时使用
语言运行时镜像       python:3.9-slim       直接运行 Python 应用
最小化镜像            alpine:3.14           追求极小的镜像体积
专用服务镜像            mysql:8.0            直接部署数据库服务

In [ ]:
3. 镜像版本管理
始终指定具体版本标签，避免使用 latest
FROM ubuntu:20.04  # ✅ 明确版本
# FROM ubuntu       # ❌ 可能随 latest 变化导致构建不稳定

In [ ]:
4. 企业私有仓库
如果有内部镜像仓库（如 Harbor、Nexus），通过地址直接引用：
FROM registry.your-company.com/base/python:3.8

In [ ]:
二、Dockerfile 位置问题
Dockerfile 的位置 完全独立于基础镜像，只需在构建时通过 docker build 指定正确的上下文路径。
即：不需要将基础镜像放到项目文件根目录，爱在哪在哪，只要docker images 能搜索到即可。
# 推荐项目结构
my-project/
├── Dockerfile       # 核心构建文件
├── .dockerignore    # 忽略不需要的文件
├── src/             # 应用程序代码
└── requirements.txt # 依赖清单
# 构建操作示例
# 在项目根目录执行（Dockerfile 所在位置）
docker build -t myapp:v1 .
--末尾的 . 表示将当前目录作为构建上下文，Docker 引擎会打包上下文中的所有文件（受 .dockerignore 约束）

In [ ]:
三、最佳实践建议

In [ ]:
1. 基础镜像选择原则
最小化：优先选择 -slim 或 -alpine 版本
    FROM python:3.9-slim  # 体积仅 120MB 左右
安全性：定期更新基础镜像版本以获取安全补丁

In [ ]:
2. 多阶段构建
适用于需要编译的场景，最终镜像仅保留运行时必要文件：
# 构建阶段
FROM golang:1.18 AS builder
WORKDIR /app
COPY . .
RUN go build -o myapp

# 运行阶段
FROM alpine:3.14
COPY --from=builder /app/myapp /usr/local/bin/
CMD ["myapp"]

In [ ]:
3. 镜像层级优化
合并同类操作：
# ✅ 合并 RUN 指令减少层数
RUN apt-get update && \
    apt-get install -y git curl && \
    apt-get clean

# ❌ 避免多个独立 RUN
RUN apt-get update
RUN apt-get install -y git

In [ ]:
4. 完整操作
# 创建项目目录
mkdir python-app && cd python-app
# 使用官方镜像
FROM ubuntu:20.04
# 设置环境
ENV DEBIAN_FRONTEND=noninteractive # 参见 4.1
RUN apt-get update && \
    apt-get install -y python3-pip && \
    apt-get clean
# 复制代码
WORKDIR /app
COPY . .
# 安装依赖
RUN pip3 install -r requirements.txt
# 启动命令
CMD ["python3", "app.py"]
# 构建镜像
docker build -t my-python-app:v1 .

In [ ]:
4.1 环境设置 DEBIAN_FRONTEND=noninteractive
这个环境变量设置用于控制 Debian/Ubuntu 系统包管理器的交互行为。
（1）作用原理
ENV DEBIAN_FRONTEND=noninteractive 是专门针对 Debian/Ubuntu 系 Linux 发行版的特殊设置，它改变了 apt-get 等包管理工具的工作方式：
默认行为：在未设置该变量时，当安装某些需要用户交互的软件包（如时区选择、许可证确认等）时，安装过程会暂停并等待用户输入。
设置后的行为，通过设置为 noninteractive：自动选择默认配置；跳过所有交互式提问；对于需要输入的情况，会自动使用默认值继续安装
（2）在 Docker 中的必要性
构建不可中断性-Docker 镜像构建过程必须是完全自动化的，无法进行人工交互
环境一致性-避免因交互选择不同导致镜像差异
构建速度-跳过等待输入环节可以加快构建过程
（3）注意事项
**作用范围：该设置仅对当前构建阶段有效，不会影响最终镜像的运行环境
**多阶段构建：如果使用多阶段构建，每个 FROM 阶段都需要重新设置
**替代方案-对于特定命令可以使用参数临时设置：
  RUN DEBIAN_FRONTEND=noninteractive apt-get install -y some-package
**副作用：某些软件包可能需要额外的自动应答配置（如使用 debconf-set-selections）

In [ ]:
4.2 基础环境最佳实践
FROM ubuntu:20.04

# 设置非交互模式
ENV DEBIAN_FRONTEND=noninteractive

# 安装带有时区配置的软件包
RUN apt-get update && \
    apt-get install -y \
    tzdata \
    && \
    ln -fs /usr/share/zoneinfo/Asia/Shanghai /etc/localtime && \
    dpkg-reconfigure --frontend noninteractive tzdata && \
    apt-get clean